<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#EDA" data-toc-modified-id="EDA-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>EDA</a></span></li><li><span><a href="#Очистка-текста" data-toc-modified-id="Очистка-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Очистка текста</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Разбиение-датасета-на-выборки-и-выедление-признаков" data-toc-modified-id="Разбиение-датасета-на-выборки-и-выедление-признаков-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Разбиение датасета на выборки и выедление признаков</a></span></li><li><span><a href="#Получение-TF-IDF" data-toc-modified-id="Получение-TF-IDF-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Получение TF-IDF</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### EDA

In [1]:
import pandas as pd
import numpy as np

import re

import nltk
import spacy

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import f1_score

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.isna().mean()

Unnamed: 0    0.0
text          0.0
toxic         0.0
dtype: float64

In [5]:
df.duplicated().sum()

0

In [6]:
len(df) == df['Unnamed: 0'].nunique()

True

In [7]:
df = df.drop('Unnamed: 0', axis=1)

**Вывод:** в датасете нет пропущенных значений и дупликатов. Также из датасета был удален столбец `Unnamed: 0`, который дублировал индекс. 

In [8]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

**Вывод:** для данных характерен дисбаланс классов.

### Очистка текста

In [9]:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [10]:
%%time
df['text'] = df['text'].apply(clear_text)

CPU times: user 4.04 s, sys: 66.7 ms, total: 4.11 s
Wall time: 4.11 s


### Лемматизация

In [11]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [12]:
%%time
nltk.download('averaged_perceptron_tagger')

df['text'] = df['text'].apply(lemmatize)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


CPU times: user 16min 59s, sys: 1min 56s, total: 18min 56s
Wall time: 18min 57s


### Разбиение датасета на выборки и выедление признаков

In [13]:
X = df.drop('toxic', axis=1)
y = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=12345)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=12345)

In [14]:
print(X_train.shape, X_test.shape, X_valid.shape, sep='\n')
print(y_train.shape, y_test.shape, y_valid.shape, sep='\n')

(63716, 1)
(47788, 1)
(47788, 1)
(63716,)
(47788,)
(47788,)


### Получение TF-IDF

In [16]:
X_train = X_train.copy()
X_test = X_test.copy()
X_valid = X_valid.copy()
y_train = y_train.copy()
y_test = y_test.copy()
y_valid = y_valid.copy()

In [17]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = tf_idf.fit(X_train['text'].values)
train_tf_idf = tf_idf.transform(X_train['text'].values)
valid_tf_idf = tf_idf.transform(X_valid['text'].values)
test_tf_idf = tf_idf.transform(X_test['text'].values)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
train_tf_idf.shape, valid_tf_idf.shape, test_tf_idf.shape

((63716, 86606), (47788, 86606), (47788, 86606))

**Вывод:** в ходе EDA убедились, что данные корректны, без дупликатов и пропусков. Лишний столбец `Unnamed: 0`, дублирующий индекс был удален. Для переменной `toxic` характерен дисбаланс классов. Однако, после преобразования TF-IDF он не страшен. Также мы очистили текст. Провели токенизацию и лемматизацию. Разбили данные на выборки в соотношении 3:1:1, выдлелил признаки. Трансформировали текстовые данные в векторы и получили TF-IDF.

## Обучение

### Логистическая регрессия

In [19]:
%%time
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words='english')),
        ("clf", LogisticRegression())
    ]
)

parametrs_grid = {
    "clf__solver": ('liblinear', 'saga','newton-cg', 'lbfgs'),
    "vect__ngram_range": ((1, 1), (1, 2)),
    "vect__norm": ("l1", "l2"),
    "clf__C": [1, 10], 
    'clf__random_state': ([12345]),
    'clf__class_weight': (['balanced'])
}

grid_search_cv = GridSearchCV(pipeline, 
                              parametrs_grid, 
                              scoring='f1', 
                              cv=3,
                              n_jobs =-1
                             )

CPU times: user 194 µs, sys: 22 µs, total: 216 µs
Wall time: 221 µs


In [21]:
%%time
grid_search_cv.fit(X_train['text'], y_train)
grid_search_cv.best_score_
grid_search_cv.best_estimator_ 

0.7708361623770603

In [22]:
%%time
log_reg = LogisticRegression()
log_reg.fit(train_tf_idf, y_train)
pred = log_reg.predict(valid_tf_idf)
print(f'F-1: {f1_score(y_valid, pred)}')

F-1: 0.7040932708148523
CPU times: user 11.4 s, sys: 19.5 s, total: 30.9 s
Wall time: 31 s


In [30]:
%%time
log_reg = LogisticRegression(C=5, class_weight='balanced', solver='saga', max_iter=200, random_state=12345)
log_reg.fit(train_tf_idf, y_train)
pred = log_reg.predict(valid_tf_idf)
print(f'F-1: {f1_score(y_valid, pred)}')

F-1: 0.7503458452457806
CPU times: user 9.64 s, sys: 0 ns, total: 9.64 s
Wall time: 9.71 s


In [33]:
pred = log_reg.predict(test_tf_idf)
print(f'F-1: {f1_score(y_test, pred)}')

F-1: 0.7357224990417786


### CatBoostClassifier

In [31]:
cb = CatBoostClassifier(verbose=100, text_features=['text'])
cb.fit(X_train, y_train)
cb_pred = cb.predict(X_valid)
print(f'F-1: {f1_score(y_valid, cb_pred)}')

Learning rate set to 0.060722
0:	learn: 0.6095904	total: 483ms	remaining: 8m 2s
100:	learn: 0.1316881	total: 42.6s	remaining: 6m 19s
200:	learn: 0.1199774	total: 1m 25s	remaining: 5m 38s
300:	learn: 0.1121457	total: 2m 8s	remaining: 4m 57s
400:	learn: 0.1069512	total: 2m 49s	remaining: 4m 13s
500:	learn: 0.1027293	total: 3m 31s	remaining: 3m 30s
600:	learn: 0.0990502	total: 4m 14s	remaining: 2m 49s
700:	learn: 0.0956789	total: 4m 58s	remaining: 2m 7s
800:	learn: 0.0930587	total: 5m 41s	remaining: 1m 24s
900:	learn: 0.0905876	total: 6m 23s	remaining: 42.1s
999:	learn: 0.0884506	total: 7m 3s	remaining: 0us
F-1: 0.7801434589547991


In [32]:
cb_pred = cb.predict(X_test)
print(f'F-1: {f1_score(y_test, cb_pred)}')

F-1: 0.772721872401093


**Вывод:** лучше всех себя показала `логистическая регрессия` и по времени и по метрике.

## Выводы

После EDA и предобработки данных, очистки текста, токенизациb и лемматизации, разбиения данных на выборки в соотношении 3:1:1, и выдлеления признаков. Трансформировали текстовые данные в векторы и получили TF-IDF. После обучения, получили лучший результат `F-1: 0.7500488949735967` у `Логистической регрессии`